#### Prerequesites:
* Define schema (make figure) 
* Define structure (make figure)
* Define schema class in python (src/models/model_schemata.py)
* Convert schema + structure into model template (models/templates/p_model.pl)

#### Procedure:
* Load data
* Adjust model script for lfi accordingly
* Create evidence file (if I don't need different evidence files, move to builmodel notebook)
* LFI

Multiple instances, 1 pst, X enzymes, one sample per instance, multiple enzymes per instance, test how many enzymes can have in one instances, how computing time depends on N p::f for 60 samples

In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
# Importing libraries
import pandas as pd
import numpy as np
import os
import random

from src.models import model_schemata as schema
from src.models import build_model as build
from src.visualization import visualize as viz

In [8]:
# os.chdir('/Users/magdalena/OneDrive - Queen Mary, University of London/bezzlab/research/projects/phospho_pi/')
os.chdir('/home/mhuebner/Desktop/bezzlab/research/projects/phospho_pi/')

#### Training/Testing

In [9]:
# reading csv from files into dict
data = {}
data['e_ksea'] = pd.read_csv('data/processed/ebdt_data/run1/e_ksea.csv')
data['p_fc'] = pd.read_csv('data/processed/ebdt_data/run1/p_fc.csv')
data['es_interaction'] = pd.read_csv('data/processed/ebdt_data/run1/es_interaction_top7enz.csv')
data['e_ksea']

,enzyme,sample,tc,value,prob,p_dec,p_inc
0,ABL1,AC220,10.0,inc,0.648536,0.001000,0.648536
1,ABL1,AT13148,10.0,inc,0.590795,0.001000,0.590795
2,ABL1,AZ20,10.0,inc,0.651697,0.001000,0.651697
3,ABL1,AZD1480,10.0,inc,0.569660,0.001000,0.569660
4,ABL1,AZD3759,10.0,dec,0.836416,0.836416,0.001000
...,...,...,...,...,...,...,...
16648,WNK2,Torin,1.0,inc,0.745643,0.001000,0.745643
16649,WNK2,Trametinib,1.0,inc,0.596578,0.001000,0.596578
16650,WNK2,U73122,1.0,dec,0.548430,0.548430,0.001000
16651,WNK2,Ulixertinib,1.0,inc,0.646407,0.001000,0.646407


Splitting data into training/testing

In [10]:
# getting sample names (union of all samples in all datasets)
samples = list(set(data['e_ksea']['sample']).union(set(data['p_fc']['sample'])))
samples.sort()
# sample x% of samples randomly without replacement with seed
random.seed(612)
train = random.sample(samples, int(len(samples)*1))
print(train)
# filter data
training_data = {}
training_data['e_ksea'] = data['e_ksea'][data['e_ksea']['sample'].isin(train)].reset_index(drop=True)
training_data['p_fc'] = data['p_fc'][data['p_fc']['sample'].isin(train)].reset_index(drop=True)

['TAK715', 'TGX221', 'CHIR99021', 'CAL101', 'Ripasudil', 'KN93', 'LY2584702', 'LY2090314', 'Ku0063794', 'SP600125', 'Ribociclib', 'U73122', 'AZD5438', 'GDC0994', 'Edelfosine', 'PF3758309', 'Amuvatinib', 'AT13148', 'AZD3759', 'Go6976', 'JNJ', 'NU7441', 'GSK2334470', 'Ipatasertib', 'AZD5363', 'LY2835219', 'Torin', 'Bosutinib', 'H89', 'DNAPK', 'GDC0941', 'FRAX486', 'CX4945', 'HS173', 'Linsitinib', 'Dabrafenib', 'AZD1480', 'BX912', 'AZ20', 'TBCA', 'Selumetinib', 'GF109203X', 'AZD6482', 'AC220', 'PH797804', 'AZD8055', 'Trametinib', 'JNK']


In [11]:
import multiprocessing
import concurrent.futures

# Define a function to process a single sample
def process_sample(sample, phosphosites, interactions, predicates, evidence_dict):
    evidence_dict[sample] = {}
    count = -1

    # Precompute data
    evid_generator_e = build.ProblogStatementGenerator(predicates['e_ksea'])
    evid_generator_p = build.ProblogStatementGenerator(predicates['p_fc'])

    for p in phosphosites:
        count += 1
        # Print count if divisible by 1000 to track progress
        if count % 100 == 0:
            print(f"{sample}: {count}")

        enzymes = interactions['enzyme'][interactions['substrate'] == p].tolist()

        evidence_e = []
        for e in enzymes:
            evidence_e.extend(evid_generator_e.generate_facts(build.EvidenceTemplate, select=[sample, e]))

        evidence_p = evid_generator_p.generate_facts(build.EvidenceTemplate, select=[sample, p])
        evidence_dict[sample][p] = evidence_e + evidence_p

    return evidence_dict

if __name__ == "__main__":
    num_processes = multiprocessing.cpu_count()  # Use all available CPU cores

    samples = list(set(training_data['e_ksea']['sample']) | set(training_data['p_fc']['sample']))
    phosphosites = list(set(training_data['p_fc']['phosphosite']))
    evidence_dict = {}

    # Create a ThreadPoolExecutor with the desired number of threads
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_processes) as executor:
        futures = []

        for sample in samples:
            # Precompute data
            # Mapping data to Problog predicates
            predicates = {}
            predicates['e_ksea'] = schema.EKseaPredicate()
            predicates['e_ksea'].add_data(training_data['e_ksea'][training_data['e_ksea']['sample'] == sample], 'enzyme', 'sample', 'value')
            predicates['p_fc'] = schema.PFoldChangePredicate()
            predicates['p_fc'].add_data(training_data['p_fc'][training_data['p_fc']['sample'] == sample], 'phosphosite', 'sample', 'value')
            future = executor.submit(process_sample, sample, phosphosites, data['es_interaction'], predicates, evidence_dict)
            futures.append(future)

        # Wait for all tasks to complete and retrieve their results
        for future in concurrent.futures.as_completed(futures):
            evidence_dict = future.result()

    # Now, evidence_dict contains the results of parallel processing using ThreadPoolExecutor

AZD6482: 0
AZD6482: 100
AZD6482: 200
Ku0063794: 0
NU7441: 0
AZD6482: 300
Ku0063794: 100
LY2835219: 0
AZD6482: 400
LY2835219: 100
LY2835219: 200
NU7441: 100
AZD8055: 0
Ku0063794: 200
TAK715: 0
Ku0063794: 300
GSK2334470: 0
AZD8055: 100
NU7441: 200
AZD6482: 500
Amuvatinib: 0
Amuvatinib: 100
Amuvatinib: 200
PF3758309: 0
Amuvatinib: 300
Linsitinib: 0
TAK715: 100
Ribociclib: 0
LY2835219: 300
Ribociclib: 100
Ku0063794: 400
Ribociclib: 200
Ribociclib: 300
Linsitinib: 100
AZ20: 0
LY2835219: 400
NU7441: 300
GSK2334470: 100PF3758309: 100

CAL101: 0
AZD8055: 200
AZ20: 100
SP600125: 0
Amuvatinib: 400
AZD3759: 0
Ku0063794: 500
TAK715: 200
Ipatasertib: 0
GF109203X: 0
TBCA: 0
AC220: 0
Linsitinib: 200
AZD6482: 600
CAL101: 100
CX4945: 0
AT13148: 0
NU7441: 400
JNK: 0
GSK2334470: 200
LY2835219: 500
AZ20: 200
Ribociclib: 400
Dabrafenib: 0
AZD3759: 100
SP600125: 100
Selumetinib: 0
TAK715: 300
PF3758309: 200
Ku0063794: 600
Amuvatinib: 500
AC220: 100
Linsitinib: 300
TBCA: 100
Ipatasertib: 100
AZD6482: 700
JNK

In [8]:
# write evidence_dict to text file: loop over samples and separate by line of hyphens
with open('models/ebdt_data/p_model/evidence_run1.pl', 'w') as f:
    for s in evidence_dict:
        for p in evidence_dict[s]:
            for e in evidence_dict[s][p]:
                f.write(e + '\n')
            f.write('--------------------' + '\n')

In [12]:
# write evidence_dict to text file: loop over samples and separate by line of hyphens
with open('models/ebdt_data/p_model/evidence_run1_training.pl', 'w') as f:
    for s in evidence_dict:
        for p in evidence_dict[s]:
            for e in evidence_dict[s][p]:
                f.write(e + '\n')
            f.write('--------------------' + '\n')

LFI

In [9]:
DIR = 'models/ebdt_data/p_model/'
max_iter = 20
# learning from interpretation
cmd = f'problog lfi {DIR}p_model_lfi_run1.pl {DIR}evidence_run1.pl -O {DIR}p_model_run1_20i.pl --logger {DIR}log_run1-20i.txt -k ddnnf -v -n {max_iter}'
os.system(cmd)

In [9]:
DIR = 'models/ebdt_data/p_model/'
# learning from interpretation
cmd = f'problog ground {DIR}p_model_lfi_exp3.pl -o {DIR}p_model_exp3_ground.pl --format pl'
os.system(cmd)

(True, 't(0.33,"CDK10","BX912")::e_activity("CDK10","BX912",inc); t(0.33,"CDK10","BX912")::e_activity("CDK10","BX912",dec).\nt(0.33,"CDK13","BX912")::e_activity("CDK13","BX912",inc); t(0.33,"CDK13","BX912")::e_activity("CDK13","BX912",dec).\nt(0.33,"CDK16","BX912")::e_activity("CDK16","BX912",inc); t(0.33,"CDK16","BX912")::e_activity("CDK16","BX912",dec).\nt(0.33,"CDK18","BX912")::e_activity("CDK18","BX912",inc); t(0.33,"CDK18","BX912")::e_activity("CDK18","BX912",dec).\nt(0.33,"CDK20","BX912")::e_activity("CDK20","BX912",inc); t(0.33,"CDK20","BX912")::e_activity("CDK20","BX912",dec).\nt(0.33,"CDK8","BX912")::e_activity("CDK8","BX912",inc); t(0.33,"CDK8","BX912")::e_activity("CDK8","BX912",dec).\nt(0.33,"CDK9","BX912")::e_activity("CDK9","BX912",inc); t(0.33,"CDK9","BX912")::e_activity("CDK9","BX912",dec).\nt(0.33,"CTDSPL","BX912")::e_activity("CTDSPL","BX912",inc); t(0.33,"CTDSPL","BX912")::e_activity("CTDSPL","BX912",dec).\n0.001::occ_dec("RB1(S807)","BX912").\n0.001::occ_dec("RB1(S8

256